# Extracting results: Part I

This tutorial showcases how results can be extracted, including how user defined templates are able to create new result
calculations.

In [1]:
import iesopt

INFO:iesopt:Setting up Julia ...
[ Info: Now using Revise
INFO:iesopt:Julia setup successful
INFO:iesopt:Importing Julia module `IESoptLib`
INFO:iesopt:Importing Julia module `IESopt`


Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


INFO:iesopt:Importing Julia module `JuMP`
INFO:iesopt:╔════════════════════════════════════════════════════════════════════════╗
INFO:iesopt:║            IESopt   «Integrated Energy System Optimization»            ║
INFO:iesopt:╟────────────────────────────────────────────────────────────────────────╢
INFO:iesopt:║   ╭────────────────────────────────────────────────────────────────╮   ║
INFO:iesopt:║   ├ authors: Stefan Strömer, Daniel Schwabeneder, and contributors │   ║
INFO:iesopt:║   ├ ©  2021: AIT Austrian Institute of Technology GmbH             │   ║
INFO:iesopt:║   ├    docs: https://ait-energy.github.io/iesopt                   │   ║
INFO:iesopt:║   ├ version: ┐                                                     │   ║
INFO:iesopt:║   │          ├─{ py  :: 1.0.0a3 }                                  │   ║
INFO:iesopt:║   │          ├─{ jl  :: 1.0.3   }                                  │   ║
INFO:iesopt:║   │          └─{ lib :: 0.2.0   }                                  │   ║
I

In [2]:
config_file = iesopt.make_example(
    "48_custom_results", dst_dir="ex_custom_results", dst_name="config"
)

INFO:iesopt:Data folder for examples already exists; NOT copying ANY contents
INFO:iesopt:Creating example ('48_custom_results') at: 'ex_custom_results/config.iesopt.yaml'
INFO:iesopt:Set write permissions for example ('ex_custom_results/config.iesopt.yaml'), and data folder ('ex_custom_results/files')


In [ ]:
model = iesopt.run(config_file)

assert model.status == iesopt.ModelStatus.OPTIMAL

## Accessing model results: Objectives

The objective of your model - after a successful solve - can be extracted using:

In [4]:
model.objective_value

981.1745152354571

It may however be the case, that you have registered multiple objective functions (which can be used for multi-objective
algorithms, or even be useful just for analysis purposes). You can get their value by their name. The default objective
is always called `total_cost` and is the only one guaranteed to always exist. We can check which objectives are registered:

In [5]:
model.results.objectives.keys()

dict_keys(['total_cost'])

And get the value of `total_cost` (which should match the one obtained from `model.objective_value` for this example):

In [6]:
model.results.objectives["total_cost"]

981.1745152354571

## Accessing model results: Variables

Three different ways to access the results of our custom storage component `storage`:

### Direct access

In [ ]:
model.results.components["storage"].exp.setpoint

array([-3.6565097 ,  3.47368421, -0.99722992,  0.94736842,  0.        ,
        0.        ,  0.        , -5.6       ,  0.        , -0.1       ,
       -1.50221607,  3.47368421,  2.63157895,  0.31578947,  0.42105263,
       -3.46149584,  0.        ,  0.84210526, -2.4       , -4.        ,
        3.57894737,  3.47368421,  0.94736842,  0.52631579])

> Accessing dual information:
> ```python
> model.results.components["grid"].con.nodalbalance__dual
> ```

### Access using `get(...)`

In [ ]:
model.results.get("component", "storage", "exp", "setpoint")

array([-3.6565097 ,  3.47368421, -0.99722992,  0.94736842,  0.        ,
        0.        ,  0.        , -5.6       ,  0.        , -0.1       ,
       -1.50221607,  3.47368421,  2.63157895,  0.31578947,  0.42105263,
       -3.46149584,  0.        ,  0.84210526, -2.4       , -4.        ,
        3.57894737,  3.47368421,  0.94736842,  0.52631579])

> Accessing dual information:
> ```python
> model.results.get("component", "grid", "con", "nodalbalance", mode="dual")
> ```

### Collective results

#### Using `to_dict(...)`

In [ ]:
results = model.results.to_dict()

results[("storage", "exp", "setpoint")]

array([-3.6565097 ,  3.47368421, -0.99722992,  0.94736842,  0.        ,
        0.        ,  0.        , -5.6       ,  0.        , -0.1       ,
       -1.50221607,  3.47368421,  2.63157895,  0.31578947,  0.42105263,
       -3.46149584,  0.        ,  0.84210526, -2.4       , -4.        ,
        3.57894737,  3.47368421,  0.94736842,  0.52631579])

> Accessing dual information:
> ```python
> results[("grid", "con", "nodalbalance__dual")]
> ```

> Note: You can filter the results returned by `to_dict(...)` in exactly the same way as when using `to_pandas(...)` (
> see below). However, this is uncommon to be useful, since you most likely want to work with tabular data anyways when
> using the filter function, which is why we skip it here.

#### Using `to_pandas(...)`

In [ ]:
df = model.results.to_pandas()

df.loc[
    (
        (df["component"] == "storage")
        & (df["fieldtype"] == "exp")
        & (df["field"] == "setpoint")
        & (df["mode"] == "primal")
    ),
    "value",
].values

array([-3.6565097 ,  3.47368421, -0.99722992,  0.94736842,  0.        ,
        0.        ,  0.        , -5.6       ,  0.        , -0.1       ,
       -1.50221607,  3.47368421,  2.63157895,  0.31578947,  0.42105263,
       -3.46149584,  0.        ,  0.84210526, -2.4       , -4.        ,
        3.57894737,  3.47368421,  0.94736842,  0.52631579])

In [ ]:
series = model.results.to_pandas(
    lambda c, t, f: c == "storage" and t == "exp" and f == "setpoint"
)
series.head()

t1   -3.656510
t2    3.473684
t3   -0.997230
t4    0.947368
t5    0.000000
Name: (storage, res, setpoint), dtype: float64

We could actually only filter for the component (`c == "storage"`), since this is the only result that it creates.

In [8]:
series = model.results.to_pandas(lambda c, t, f: c == "storage")
series.head()

t1   -3.656510
t2    3.473684
t3   -0.997230
t4    0.947368
t5    0.000000
Name: (storage, res, setpoint), dtype: float64

This may however be dangerous, since a similar call

```python
model.results.to_pandas(lambda c,t,f: c == "grid")
```

would then suddenly return a `pd.DataFrame`, since it contains two different results (try it out!) linked to the
component `grid`.

> Accessing dual information (part I):
>
> ```python
> model.results.to_pandas(lambda c,t,f: c == "grid" and t == "con")
> ```
>
> This works, since the model only contains a single result linked to constraints of the component `grid`. However, this
> may again be dangerous, which is why you could instead make use of something like
>
> ```python
> df = model.results.to_pandas()
> df[df["mode"] == "dual"]
> ```
>
> _Note that this extracts ALL dual results, not only those for the component used above, but again `to_pandas(...)` is
> mostly there to extract more than one result at the same time (we cover "Which way should I use below?")._

---

You may now wonder - since it all looks the same - what `to_pandas(...)` could be useful for. It's main usage is
extracting more than one result in one call:

In [28]:
df = model.results.to_pandas(field_types="exp", orientation="wide")
df.head(5)

,storage.storage,demand,grid,generator
,exp,exp,exp,exp
,injection,value,injection,out_electricity
t1,3.473684,4.0,4.440892e-16,7.65651
t2,-3.473684,4.0,-1.110223e-16,0.70000
t3,0.947368,4.0,2.220446e-16,4.99723
t4,-0.947368,4.0,0.000000e+00,3.10000
t5,0.000000,4.0,0.000000e+00,4.00000


### Which result extraction should I use?

As a basic guide you can use the following logic to decide how to extract results. Are you:

1. Looking for a single result of a component? Extract it similar to `model.results.components["storage"].exp.setpoint`.
2. Looking for multiple results of a component (e.g., all objective terms created by a single `Unit`), or similar results of multiple components (e.g., electricity generation of all generators)? Make use of `to_pandas(...)`, applying a specific filter, and either using `orientation = "long"` (the default), or `orientation = "wide"`.

---

**Advanced usage:** _Looking for a single result of a component at a time, but doing it repeatedly for a single run (= extracting a single result from component `A`, then one from component `B`, and  so on)?_

Then use the `to_dict(...)` function and then extract your results similar to `model.to_dict()[("storage", "exp", "setpoint")]`. Compared to (1.) this has the advantage of caching results during the first call to `to_dict(...)`, and being able to only extract specific results if correctly filtered. **Pay attention to why, when, and how you use this, since improper usage may be way slower than directly accessing your results as explained above.

## Loading results from file

The example that we have used until now, does not write any results to a file. To load results from file in later
program runs, we therefore need to enable writing to a file and re-solve the model. 

For that, edit the top-level config file, and change

```yaml
config:
  # ...
  results:
    enabled: true
    memory_only: true
```

to

```yaml
config:
  # ...
  results:
    enabled: true
    memory_only: false       # <-- change here!

Now run the model once more to create the result output file

In [ ]:
model = iesopt.run("ex_custom_results/config.iesopt.yaml")

An IESopt model:
	name: CustomResults
	solver: HiGHS
	
	96 variables, 170 constraints
	status: OPTIMAL

This will create an IESopt result file `SomeScenario.iesopt.result.jld2` inside the
`ex_custom_results/out/CustomResults/` folder, which contains all results. This can be used to analyse results at a
later time. To prevent losing information it tries to extract _**all**_ results - which may be time intensive, but
ensures that you do not forget to extract something, to only realise later that you miss it.

We can now load this file using

In [17]:
results = iesopt.Results(
    file="ex_custom_results/out/CustomResults/SomeScenario.iesopt.result.jld2"
)

Now, you can use exactly the same code that we have already walked through, c.f. {ref}`Accessing model results: Variables`, just by
replacing every access to `model.results` by `results`.

### File results: Direct access

In [ ]:
# instead of
#   `model.results.components["storage"].exp.setpoint`
# we now use:

results.components["storage"].exp.setpoint

array([-3.6565097 ,  3.47368421, -0.99722992,  0.94736842,  0.        ,
        0.        ,  0.        , -5.6       ,  0.        , -0.1       ,
       -1.50221607,  3.47368421,  2.63157895,  0.31578947,  0.42105263,
       -3.46149584,  0.        ,  0.84210526, -2.4       , -4.        ,
        3.57894737,  3.47368421,  0.94736842,  0.52631579])

### File results: Access using `get(...)`

In [ ]:
results.get("component", "storage", "exp", "setpoint")

array([-3.6565097 ,  3.47368421, -0.99722992,  0.94736842,  0.        ,
        0.        ,  0.        , -5.6       ,  0.        , -0.1       ,
       -1.50221607,  3.47368421,  2.63157895,  0.31578947,  0.42105263,
       -3.46149584,  0.        ,  0.84210526, -2.4       , -4.        ,
        3.57894737,  3.47368421,  0.94736842,  0.52631579])

### File results: Collective results

#### File results: Using `to_dict(...)`

In [ ]:
result_dict = results.to_dict()
result_dict[("storage", "exp", "setpoint")]

array([-3.6565097 ,  3.47368421, -0.99722992,  0.94736842,  0.        ,
        0.        ,  0.        , -5.6       ,  0.        , -0.1       ,
       -1.50221607,  3.47368421,  2.63157895,  0.31578947,  0.42105263,
       -3.46149584,  0.        ,  0.84210526, -2.4       , -4.        ,
        3.57894737,  3.47368421,  0.94736842,  0.52631579])

#### File results: Using `to_pandas(...)`

In [23]:
df = results.to_pandas(field_types="exp", orientation="wide")
df.head()

,storage.storage,demand,grid,generator
,exp,exp,exp,exp
,injection,value,injection,out_electricity
t1,3.473684,4.0,4.440892e-16,7.65651
t2,-3.473684,4.0,-1.110223e-16,0.70000
t3,0.947368,4.0,2.220446e-16,4.99723
t4,-0.947368,4.0,0.000000e+00,3.10000
t5,0.000000,4.0,0.000000e+00,4.00000


## Calling into Julia

You should probably never need the following, but you can also manually access the `results` data `Struct` inside the
Julia model to extract some results. For an optimized model (not for results loaded from a file!), this could be done
using

In [ ]:
my_result = (
    model.core.ext[iesopt.Symbol("iesopt")].results.components["storage"].exp.setpoint
)

Observe that

In [22]:
type(my_result)

juliacall.VectorValue

which shows that the other modes of result extraction take care of a proper Julia-to-Python conversion for you already.
Further, you can then

In [23]:
my_result[:5]

5-element view(::Vector{Float64}, 1:1:5) with eltype Float64:
 -3.656509695290859
  3.473684210526316
 -0.997229916897507
  0.9473684210526315
  0.0

which, as you see, returns an actual `view` into the `Vector{Float64}`, indexed using the Julia range `1:1:5` (given by
the Python range `:5`). But

In [24]:
my_result[0:5]

5-element view(::Vector{Float64}, 1:1:5) with eltype Float64:
 -3.656509695290859
  3.473684210526316
 -0.997229916897507
  0.9473684210526315
  0.0

makes it clear, that the wrapper we use automatically translates between 0-based (Python) and 1-based (Julia) indexing,
which may become confusing and error-prone when thinking about a Julia `Vector` but accessing the first entry using

In [25]:
my_result[0]

-3.656509695290859

> Accessing dual information (part I):
>
> ```python
> model.core.ext[iesopt.Symbol("iesopt")].results.components["grid"].con.nodalbalance__dual
> ```